<a href="https://colab.research.google.com/github/Krishnaa0910/Image-Deblurring/blob/main/Deblur_Hybrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

dataset_path = r'D:\deblur\GoPro'

def load_images(dataset_path, img_size=(256, 256)):
    blur_images = []
    sharp_images = []

    blur_path = os.path.join(dataset_path, "train", "input/input")
    sharp_path = os.path.join(dataset_path, "train", "target/target")

    for img in os.listdir(blur_path):
        blur_img = cv2.imread(os.path.join(blur_path, img))
        if blur_img is None:
            continue
        blur_img = cv2.resize(blur_img, img_size)
        blur_img = blur_img.astype(np.float32) / 255.0
        blur_images.append(blur_img)

        sharp_img = cv2.imread(os.path.join(sharp_path, img))
        if sharp_img is None:
            continue
        sharp_img = cv2.resize(sharp_img, img_size)
        sharp_img = sharp_img.astype(np.float32) / 255.0
        sharp_images.append(sharp_img)

    return np.array(blur_images), np.array(sharp_images)

# Loading images
blur_images, sharp_images = load_images(dataset_path)



In [ ]:
def srn_generator(input_shape=(256, 256, 3)):
    inputs = Input(shape=input_shape)

    # Encoding layers
    x = Conv2D(64, kernel_size=5, strides=1, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Several more convolutional layers for downsampling
    for _ in range(3):
        x = Conv2D(128, kernel_size=5, strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

    # Decoding layers
    for _ in range(3):
        x = Conv2DTranspose(128, kernel_size=5, strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

    outputs = Conv2D(3, kernel_size=5, strides=1, padding='same', activation='tanh')(x)

    return Model(inputs, outputs)

generator = srn_generator()
generator.summary()

def build_discriminator(input_shape=(256, 256, 3)):
    inputs = Input(shape=input_shape)

    x = Conv2D(64, kernel_size=5, strides=2, padding='same')(inputs)
    x = Activation('relu')(x)

    for _ in range(3):
        x = Conv2D(128, kernel_size=5, strides=2, padding='same')(x)
        x = Activation('relu')(x)

    x = Conv2D(1, kernel_size=5, strides=1, padding='same')(x)
    outputs = Activation('sigmoid')(x)

    return Model(inputs, outputs)

discriminator = build_discriminator()
discriminator.summary()

def build_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = Input(shape=(256, 256, 3))
    deblurred_image = generator(gan_input)
    gan_output = discriminator(deblurred_image)
    gan = Model(gan_input, gan_output)
    return gan

gan = build_gan(generator, discriminator)
gan.summary()

# Compile the models
optimizer = Adam(0.0002, 0.5)
generator.compile(loss='mse', optimizer=optimizer)
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)
gan.compile(loss='binary_crossentropy', optimizer=optimizer)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 64)      4864      
                                                                 
 batch_normalization (BatchN  (None, 256, 256, 64)     256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 256, 256, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 128)     204928    
                                                                 
 batch_normalization_1 (Batc  (None, 128, 128, 128)    512       
 hNormalization)                                             

In [ ]:

epochs = 101
batch_size = 8
real_label = np.ones((batch_size, 16, 16, 1))
fake_label = np.zeros((batch_size, 16, 16, 1))

for epoch in range(epochs):
    # Training Discriminator
    idx = np.random.randint(0, blur_images.shape[0], batch_size)
    real_imgs = sharp_images[idx]
    blur_imgs = blur_images[idx]

    fake_imgs = generator.predict(blur_imgs)

    d_loss_real = discriminator.train_on_batch(real_imgs, real_label)
    d_loss_fake = discriminator.train_on_batch(fake_imgs, fake_label)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train Generator
    g_loss = gan.train_on_batch(blur_imgs, real_label)

    if epoch % 10 == 0:  # Prints every 10 epochs
        print(f"{epoch} [D loss: {d_loss}] [G loss: {g_loss}]")


1/1 [==============================] - 4s 4s/step
0 [D loss: 0.6982899308204651] [G loss: 0.6979234218597412]
1/1 [==============================] - 3s 3s/step
10 [D loss: 0.6981483399868011] [G loss: 0.5006316900253296]
1/1 [==============================] - 4s 4s/step
20 [D loss: 0.7015767693519592] [G loss: 0.4856358766555786]
1/1 [==============================] - 3s 3s/step
30 [D loss: 0.7052323818206787] [G loss: 0.47640711069107056]
1/1 [==============================] - 3s 3s/step
40 [D loss: 0.7104310095310211] [G loss: 0.46955791115760803]
1/1 [==============================] - 3s 3s/step
50 [D loss: 0.7154340147972107] [G loss: 0.46480605006217957]
1/1 [==============================] - 3s 3s/step
60 [D loss: 0.7219273746013641] [G loss: 0.46182262897491455]
1/1 [==============================] - 3s 3s/step
70 [D loss: 0.7286078929901123] [G loss: 0.4602805972099304]
1/1 [==============================] - 4s 4s/step
80 [D loss: 0.7369453608989716] [G loss: 0.4566402137279510

In [ ]:
def srn_generator(input_shape=(256, 256, 3)):
    inputs = Input(shape=input_shape)

    # Encoding layers
    x = Conv2D(64, kernel_size=5, strides=1, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Several more convolutional layers for downsampling
    for _ in range(3):
        x = Conv2D(128, kernel_size=5, strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

    # Decoding layers
    for _ in range(3):
        x = Conv2DTranspose(128, kernel_size=5, strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

    outputs = Conv2D(3, kernel_size=5, strides=1, padding='same', activation='tanh')(x)

    return Model(inputs, outputs)

generator = srn_generator()
generator.summary()


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d_10 (Conv2D)          (None, 256, 256, 64)      4864      
                                                                 
 batch_normalization_7 (Batc  (None, 256, 256, 64)     256       
 hNormalization)                                                 
                                                                 
 activation_12 (Activation)  (None, 256, 256, 64)      0         
                                                                 
 conv2d_11 (Conv2D)          (None, 128, 128, 128)     204928    
                                                                 
 batch_normalization_8 (Batc  (None, 128, 128, 128)    512       
 hNormalization)                                           

In [ ]:
def build_discriminator(input_shape=(256, 256, 3)):
    inputs = Input(shape=input_shape)

    x = Conv2D(64, kernel_size=5, strides=2, padding='same')(inputs)
    x = Activation('relu')(x)

    for _ in range(3):
        x = Conv2D(128, kernel_size=5, strides=2, padding='same')(x)
        x = Activation('relu')(x)

    x = Conv2D(1, kernel_size=5, strides=1, padding='same')(x)
    outputs = Activation('sigmoid')(x)

    return Model(inputs, outputs)

discriminator = build_discriminator()
discriminator.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d_15 (Conv2D)          (None, 128, 128, 64)      4864      
                                                                 
 activation_19 (Activation)  (None, 128, 128, 64)      0         
                                                                 
 conv2d_16 (Conv2D)          (None, 64, 64, 128)       204928    
                                                                 
 activation_20 (Activation)  (None, 64, 64, 128)       0         
                                                                 
 conv2d_17 (Conv2D)          (None, 32, 32, 128)       409728    
                                                                 
 activation_21 (Activation)  (None, 32, 32, 128)       0   

In [ ]:
def build_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = Input(shape=(256, 256, 3))
    deblurred_image = generator(gan_input)
    gan_output = discriminator(deblurred_image)
    gan = Model(gan_input, gan_output)
    return gan

gan = build_gan(generator, discriminator)
gan.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 model_3 (Functional)        (None, 256, 256, 3)       2271363   
                                                                 
 model_4 (Functional)        (None, 16, 16, 1)         1032449   
                                                                 
Total params: 3,303,812
Trainable params: 2,269,699
Non-trainable params: 1,034,113
_________________________________________________________________
